In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
import os
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 22.8 ms, sys: 7.98 ms, total: 30.8 ms
Wall time: 182 ms


### equivalent reflectivity factor

In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

### convert equivalent reflectivity factor to Iris cube

In [4]:
%%time
ERF = equivalent_reflectivity_factor.to_iris()
ERF

CPU times: user 3.99 s, sys: 253 ms, total: 4.24 s
Wall time: 4.5 s


Equivalent Reflectivity Factor (dBZ),time,projection_y_coordinate,projection_x_coordinate
Shape,301,400,400
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x


In [5]:
%%time
# Determine temporal and spatial sampling of the input data:
#grid_spacing = 1km... but tobac uses meters... 1000m = 1km
#time_spacing = 5 minutes time resolution... tobac uses seconds..... 
#since our time_spacing is 5 min, we get our time spacing in seconds.. if 60 sec = 1 min? then 5 mins = 300s... so time-spacing is 300
dxy,dt=tobac.utils.get_spacings(ERF, grid_spacing=1000, time_spacing=300)  #tobac detect it by default
dxy, dt 

CPU times: user 47 µs, sys: 5 µs, total: 52 µs
Wall time: 54.8 µs


(1000, 300)

# DETECTION FEATURE

In [6]:
%%time
# threshold = np.arange(5, 20.1, 1)
threshold = [20]
parameters_features = {}
parameters_features['target'] = 'maximum'
parameters_features['threshold'] = threshold
parameters_features['n_min_threshold'] = 0 #set to zero or one always; 
parameters_features['n_erosion_threshold'] = 0 #another filtering/smoothing method.
parameters_features['position_threshold'] ='weighted_diff'
parameters_features['sigma_threshold'] = 1 #smoothing data
# parameters_features['min_distance'] = 15

# Using 'center' here outputs the feature location as the arithmetic center of the detected feature
Features = tobac.feature_detection_multithreshold(field_in=ERF, dxy=dxy, **parameters_features)

CPU times: user 3.51 s, sys: 16.1 ms, total: 3.53 s
Wall time: 3.95 s


In [7]:
%%time
Features.head()

CPU times: user 103 µs, sys: 7 µs, total: 110 µs
Wall time: 113 µs


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude
0,0,1,115.000000,211.000000,1,20,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.175000e+06,128000.000000,70.390411,13.368005
1,0,2,133.423757,399.000000,2,20,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193424e+06,316000.000000,70.050984,18.198025
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810
3,0,4,146.000000,380.000000,1,20,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.206000e+06,297000.000000,69.961479,17.667786
4,0,5,159.000000,397.790550,2,20,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.219000e+06,314790.549960,69.822067,18.074186


In [8]:
Features.to_csv('../saved-files/threshold-20/Features-20.csv', index=False)

# SEGMENTATION

In [9]:
%%time
# Keyword arguments for the segmentation step:
parameters_segmentation={}
parameters_segmentation['target']='maximum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=20

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [10]:
%%time
# Perform segmentation and save results to files:
Mask_ERF, Features_ERF = tobac.segmentation_2D(Features,ERF,dxy,**parameters_segmentation)

CPU times: user 8.25 s, sys: 317 ms, total: 8.57 s
Wall time: 9.1 s


In [11]:
type(Mask_ERF)

iris.cube.Cube

In [12]:
iris.save(Mask_ERF, '../saved-files/threshold-20/Mask_ERF_iris-20.nc')

In [13]:
%%time
Mask_ERF

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 4.53 µs


Segmentation Mask (1),time,projection_y_coordinate,projection_x_coordinate
Shape,301,400,400
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x


In [14]:
%%time
# Convert the segmentation data from iris cube to DataArray
segmented_data = xr.DataArray.from_iris(Mask_ERF)
segmented_data

CPU times: user 3.22 ms, sys: 0 ns, total: 3.22 ms
Wall time: 3.23 ms


<xarray.DataArray 'segmentation_mask' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=int32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
    lat      (Yc, Xc) float32 ...
    lon      (Yc, Xc) float32 ...
Attributes:
    long_name:  segmentation_mask

In [15]:
segmented_data.to_netcdf('../saved-files/threshold-20/segmented_ERF-xr-20.nc')

# TRAJECTORY LINKING

In [16]:
%%time
# keyword arguments for linking step
parameters_linking={}
parameters_linking['v_max']=20
parameters_linking['stubs']=1
parameters_linking['order']=1
parameters_linking['extrapolate']=0 
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=15
parameters_linking['method_linking']= 'predict'
# parameters_linking['time_cell_min'] = 10

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [17]:
%%time
# Track=tobac.linking_trackpy(Features, ERF, dt=dt, dxy=dxy, **parameters_linking)
Track = tobac.linking_trackpy(Features, ERF, dt=dt, dxy=dxy, **parameters_linking)

Frame 300: 22 trajectories present.
CPU times: user 2.25 s, sys: 15.9 ms, total: 2.27 s
Wall time: 2.79 s


In [18]:
Track.to_csv('../saved-files/threshold-20/Track-20.csv', index=False)

In [19]:
# latA = 69.141281 #latitude of COMBLE site
# lonA = 15.684166-1 #longitude of COMBLE site -1

<h1 style="color:red;">TRACKED INFO</h1>

In [20]:
Track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,115.000000,211.000000,1,20,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.175000e+06,128000.000000,70.390411,13.368005,1,0 days 00:00:00
1,0,2,133.423757,399.000000,2,20,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193424e+06,316000.000000,70.050984,18.198025,2,0 days 00:00:00
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00
3,0,4,146.000000,380.000000,1,20,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.206000e+06,297000.000000,69.961479,17.667786,4,0 days 00:00:00
4,0,5,159.000000,397.790550,2,20,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.219000e+06,314790.549960,69.822067,18.074186,5,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15538,300,18,259.974309,323.199741,5,20,15539,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.319974e+06,240199.740987,68.991524,15.911087,3843,0 days 00:05:00
15539,300,19,272.389563,284.000000,2,20,15540,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.332390e+06,201000.000000,68.912575,14.925450,3845,0 days 00:00:00
15540,300,20,274.932709,290.064315,17,20,15541,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.334933e+06,207064.314866,68.884654,15.067793,3844,0 days 00:05:00
15541,300,21,287.000000,376.000000,1,20,15542,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.347000e+06,293000.000000,68.692009,17.116003,3817,0 days 00:30:00


# Sort Tracked info based on cell_id and time

In [21]:
%%time
track = Track.sort_values(['cell', 'time_cell'])
track = track.reset_index(drop=True)
track.head()

CPU times: user 5.27 ms, sys: 0 ns, total: 5.27 ms
Wall time: 5.28 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,115.000000,211.000000,1,20,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.175000e+06,128000.000000,70.390411,13.368005,1,0 days 00:00:00
1,0,2,133.423757,399.000000,2,20,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193424e+06,316000.000000,70.050984,18.198025,2,0 days 00:00:00
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00
3,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00
4,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00


In [22]:
track.to_csv('../saved-files/threshold-20/track-reset-20.csv', index=False)

In [23]:
# track = pd.read_csv('saved-files/track-reset.csv')

In [24]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,115.000000,211.000000,1,20,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.175000e+06,128000.000000,70.390411,13.368005,1,0 days 00:00:00
1,0,2,133.423757,399.000000,2,20,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193424e+06,316000.000000,70.050984,18.198025,2,0 days 00:00:00
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00
3,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00
4,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00


<h1 style="color:red;  text-align: center;">END OF TRACK</h1>